In [4]:
from urllib.request import urlretrieve
import zipfile, os
import numpy as np
import matplotlib.pyplot as pyplot
%matplotlib inline  

URM_file = open(r"C:\Users\aless\PYTHON_NOTEBOOKS\RECOMMENDER_SYSTEMS\movies100krates\ratings.csv", 'r')

def rowSplit (rowString):
    
    split = rowString.split(",")
    split[3] = split[3].replace("\n","")
    
    split[0] = int(split[0])
    split[1] = int(split[1])
    split[2] = float(split[2])
    split[3] = int(split[3])
    
    result = tuple(split)
    
    return result


URM_file.seek(0)
URM_tuples = []
URM_file.readline()#questa è importante di cristo c'è riga di introduzione
for line in URM_file:
    URM_tuples.append(rowSplit (line))

userList, itemList, ratingList, timestampList = zip(*URM_tuples)

userList = list(userList)
itemList = list(itemList)
ratingList = list(ratingList)
timestampList = list(timestampList)

def list_ID_stats(ID_list, label):
    
    min_val = min(ID_list)
    max_val = max(ID_list)
    unique_val = len(set(ID_list))
    missing_val = 1 - unique_val/(max_val - min_val)

    print("{} data, ID: min {}, max {}, unique {}, missig {:.2f} %".format(label, min_val, max_val, unique_val, missing_val*100))

    
    
list_ID_stats(userList, "User")
list_ID_stats(itemList, "Item")

import scipy.sparse as sps

URM_all = sps.coo_matrix((ratingList, (userList, itemList)))
URM_all = URM_all.tocsr()
#QUESTO E' IL PREWORKOUT 

#film in ordine di ID
FILM_file = open(r"C:\Users\aless\PYTHON_NOTEBOOKS\RECOMMENDER_SYSTEMS\movies100krates\movies.csv", 'r',encoding="utf8")

def rowSplit (rowString):
    
    split = rowString.split(",")
    split[2] = split[2].replace("\n","")
    
    split[0] = int(split[0])#id
    split[1] = str(split[1])#title
    split[2] = str(split[2])#genres
    
    result = tuple(split)
    
    return result


FILM_file.seek(0)
FILM_tuples = []
FILM_file.readline()#questa è importante di cristo c'è riga di introduzione
for line in FILM_file:
    FILM_tuples.append(rowSplit (line))

FILM_list=np.array(FILM_tuples)
print(len(FILM_list))#stampa quanti film ho diversi

User data, ID: min 1, max 610, unique 610, missig -0.16 %
Item data, ID: min 1, max 193609, unique 9724, missig 94.98 %
9742


In [5]:
#---qui cambia per qualche motivo e sclera

#Train/test split

from Notebooks_utils.data_splitter import train_test_holdout
from Notebooks_utils.evaluation_function import evaluate_algorithm
from Base.Similarity.Compute_Similarity_Python import Compute_Similarity_Python

URM_train, URM_test = train_test_holdout(URM_all, train_perc = 0.8)
#print(URM_train)


# Item based collaborative filtering:

In [20]:
class ItemCFKNNRecommender(object):
    
    def __init__(self, URM):
        self.URM = URM
        
            
    def fit(self, topK=50, shrink=100, normalize=True, similarity="cosine"):
        
        similarity_object = Compute_Similarity_Python(self.URM, shrink=shrink, 
                                                  topK=topK, normalize=normalize, 
                                                  similarity = similarity)
        
        self.W_sparse = similarity_object.compute_similarity()

        
    def recommend(self, user_id, at=None, exclude_seen=True):
        # compute the scores using the dot product
        user_profile = self.URM[user_id]
        scores = user_profile.dot(self.W_sparse).toarray().ravel()

        if exclude_seen:
            scores = self.filter_seen(user_id, scores)

        # rank items
        ranking = scores.argsort()[::-1]
            
        return ranking[:at]
    
    
    def filter_seen(self, user_id, scores):

        start_pos = self.URM.indptr[user_id]
        end_pos = self.URM.indptr[user_id+1]

        user_profile = self.URM.indices[start_pos:end_pos]
        
        scores[user_profile] = -np.inf

        return scores
#----------------------------------------------------------------    
    
recommender = ItemCFKNNRecommender(URM_train)
recommender.fit(shrink=0.0, topK=50)


print("La velocità con cui lavoro è:")

import time

n_users_to_test = len(set(userList))#setto numero di users x cui faccio reccomendation

start_time = time.time()

for user_id in range(n_users_to_test):
    recommender.recommend(user_id, at=5)#at è numero items che raccomando!
    
end_time = time.time()

print(" {:.2f} usr/sec".format(n_users_to_test/(end_time-start_time)))


In [4]:
print("Parameter tuning")
#x_tick = [10, 50, 100, 200, 500]
x_tick=[100] #è il migliore sennò ci metto 100 anni ogni volta che runno
MAP_per_k = []

for topK in x_tick:
    
    recommender = ItemCFKNNRecommender(URM_train)
    recommender.fit(shrink=0.0, topK=topK)
    
    result_dict = evaluate_algorithm(URM_test, recommender)
    MAP_per_k.append(result_dict["MAP"])
    
pyplot.plot(x_tick, MAP_per_k)
pyplot.ylabel('MAP')
pyplot.xlabel('TopK')
pyplot.title("Se ho un solo K ovviamente non avrò il grafico")
pyplot.show()

Parameter tuning
Similarity column 7700 (  4 % ), 256.55 column/sec, elapsed time 0.50 min


KeyboardInterrupt: 

In [8]:
print("Provo a vedere per un certo user che items mi consiglia per curiosità")
user_id=1
recommender = ItemCFKNNRecommender(URM_train)
recommender.fit(shrink=0.0, topK=100)#100 da sopra vedo che è il migliore
#faccio 1 giro di recommend per ogni utente in teoria
raccomandazioni=np.array(recommender.recommend(user_id, at=5))#at è numero items che raccomando!
print("Raccomandazioni per utente con id "+str(user_id)+": "+str(raccomandazioni))

#adesso scopro i titoli
data=FILM_list
#print(data)
films=[]
# index of rows matching the conditions
for x in data:
    if x[0] in raccomandazioni:
        films.append(x[:2])#mi dice code e titolo senza gozzoviglie dopo
print(films)

Provo a vedere per un certo user che items mi consiglia per curiosità
Similarity column 7400 (  4 % ), 245.47 column/sec, elapsed time 0.50 min
Similarity column 15100 (  8 % ), 250.17 column/sec, elapsed time 1.01 min
Similarity column 22700 ( 12 % ), 251.14 column/sec, elapsed time 1.51 min
Similarity column 30400 ( 16 % ), 252.43 column/sec, elapsed time 2.01 min
Similarity column 37700 ( 19 % ), 250.04 column/sec, elapsed time 2.51 min
Similarity column 44500 ( 23 % ), 245.80 column/sec, elapsed time 3.02 min
Similarity column 52100 ( 27 % ), 246.85 column/sec, elapsed time 3.52 min
Similarity column 59900 ( 31 % ), 248.09 column/sec, elapsed time 4.02 min
Similarity column 67600 ( 35 % ), 248.99 column/sec, elapsed time 4.52 min
Similarity column 75400 ( 39 % ), 249.77 column/sec, elapsed time 5.03 min
Similarity column 83100 ( 43 % ), 250.35 column/sec, elapsed time 5.53 min
Similarity column 90900 ( 47 % ), 250.94 column/sec, elapsed time 6.04 min
Similarity column 98800 ( 51 % 

## User base collaborative filtering
# La main diff sta che nella similarity qui ci metto la URM trasposta


In [1]:
class UserKNNCFRecommender(object):
    
    def __init__(self, URM):
        self.URM = URM
        
            
    def fit(self, topK=50, shrink=100, normalize=True, similarity="cosine"):
        
        similarity_object = Compute_Similarity_Python(self.URM.T, shrink=shrink, 
                                                  topK=topK, normalize=normalize, 
                                                  similarity = similarity)
        
        self.W_sparse = similarity_object.compute_similarity()

        
    def recommend(self, user_id, at=None, exclude_seen=True):
        # compute the scores using the dot product
        
        scores = self.W_sparse[user_id, :].dot(self.URM).toarray().ravel()

        if exclude_seen:
            scores = self.filter_seen(user_id, scores)

        # rank items
        ranking = scores.argsort()[::-1]
            
        return ranking[:at]
    
    
    def filter_seen(self, user_id, scores):

        start_pos = self.URM.indptr[user_id]
        end_pos = self.URM.indptr[user_id+1]

        user_profile = self.URM.indices[start_pos:end_pos]
        
        scores[user_profile] = -np.inf

        return scores

# Raccomando a 5 utenti alcuni filmz
(con user based CF)

In [22]:
recommender = UserKNNCFRecommender(URM_train)
recommender.fit(shrink=0.0, topK=50)#è questo quello bello lento!!

import time

n_users_to_test = 5

start_time = time.time()#BUONE METRICHE PER IL TEMPO QUI
data=FILM_list

for user_id in range(n_users_to_test):
    rec=np.array(recommender.recommend(user_id+1, at=5))
    #print("Raccomandazioni per utente con id "+str(user_id)+": "+str(rec))
    films=[]
    for x in data:
        if x[0] in rec:
            films.append(x[:2])
    print("raccomandazioni per utente "+str(user_id+1)+" : "+str(films))
    print("\n")
end_time = time.time()

print("Reasonable implementation speed is {:.2f} usr/sec".format(n_users_to_test/(end_time-start_time)))


Similarity column 611 ( 100 % ), 961.66 column/sec, elapsed time 0.01 min
raccomandazioni per utente 1 : [(480, 'Jurassic Park (1993)'), (589, 'Terminator 2: Judgment Day (1991)'), (1200, 'Aliens (1986)'), (1210, 'Star Wars: Episode VI - Return of the Jedi (1983)'), (1240, '"Terminator')]


raccomandazioni per utente 2 : [(356, 'Forrest Gump (1994)'), (2571, '"Matrix'), (58559, '"Dark Knight'), (89745, '"Avengers'), (105504, 'Captain Phillips (2013)')]


raccomandazioni per utente 3 : [(222, 'Circle of Friends (1995)'), (898, '"Philadelphia Story'), (1019, '"20'), (2150, '"Gods Must Be Crazy'), (59784, 'Kung Fu Panda (2008)')]


raccomandazioni per utente 4 : [(593, '"Silence of the Lambs'), (858, '"Godfather'), (1230, 'Annie Hall (1977)'), (1252, 'Chinatown (1974)'), (2064, 'Roger & Me (1989)')]


raccomandazioni per utente 5 : [(318, '"Shawshank Redemption'), (356, 'Forrest Gump (1994)'), (454, '"Firm'), (480, 'Jurassic Park (1993)'), (590, 'Dances with Wolves (1990)')]


Reasonable 

### Parameter tuning